In [ ]:
# Cell 1: Setup and Imports
import warnings
warnings.filterwarnings('ignore')

import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import json
import time
from dataclasses import dataclass
from typing import Dict, List, Tuple
from collections import defaultdict, deque
import seaborn as sns

# Set style for better plots
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
np.random.seed(42)

print("✅ Setup complete!")
print(f"NetworkX version: {nx.__version__}")

In [ ]:
# Cell 2: Load OptiMoE Core Implementation
# Paste the core implementation script here or import it
from optimoe_networkx_core import *



# Cell 3: Quick Test Run
"""Quick test to verify everything works"""
print("Running quick test simulation...")

# Create small test config
test_sim = OptiMoENetworkSimulation()
test_sim.moe_config.num_nodes = 16  # Small for quick test
test_sim.moe_config.num_experts = 32

# Run mini experiment
test_baseline = test_sim.run_baseline_comparison(num_iterations=6)
test_optimoe = test_sim.run_optimoe_experiment(num_iterations=30)

print(f"✅ Test completed! OptiMoE made {sum(r['reconfigure'] for r in test_optimoe)} topology switches")


In [ ]:
from optimoe_networkx_core import OptiMoENetworkSimulation

# Cell 4: Main Experiment Runner
"""Run full OptiMoE experiment with visualization"""

def run_full_experiment(num_nodes=64, num_iterations=100):
    """Run complete OptiMoE experiment"""

    print(f"Starting OptiMoE Experiment")
    print(f"Nodes: {num_nodes}, Iterations: {num_iterations}")
    print("="*50)

    # Initialize simulation
    sim = OptiMoENetworkSimulation()
    sim.moe_config.num_nodes = num_nodes
    sim.moe_config.num_experts = num_nodes * 8

    # Run baseline
    print("\n📊 Running baseline experiments...")
    start_time = time.time()
    baseline_results = sim.run_baseline_comparison(num_iterations=30)
    baseline_time = time.time() - start_time

    # Run OptiMoE
    print("\n🔄 Running OptiMoE dynamic adaptation...")
    start_time = time.time()
    optimoe_results = sim.run_optimoe_experiment(num_iterations=num_iterations)
    optimoe_time = time.time() - start_time

    print(f"\n⏱️ Timing: Baseline={baseline_time:.1f}s, OptiMoE={optimoe_time:.1f}s")

    return baseline_results, optimoe_results, sim

# Run the experiment
baseline_results, optimoe_results, sim = run_full_experiment(num_nodes=64, num_iterations=70)


In [ ]:
# Cell 5: Results Analysis and Visualization
"""Comprehensive results visualization"""

def visualize_optimoe_results(baseline_results, optimoe_results):
    """Create comprehensive visualization"""

    fig = plt.figure(figsize=(16, 10))
    gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

    # 1. Baseline Comparison
    ax1 = fig.add_subplot(gs[0, 0])
    baseline_data = []
    baseline_labels = []
    for topo, results in baseline_results.items():
        latencies = [r['average_latency'] for r in results]
        baseline_data.append(latencies)
        baseline_labels.append(topo)

    bp = ax1.boxplot(baseline_data, labels=baseline_labels, patch_artist=True)
    for patch, color in zip(bp['boxes'], ['lightblue', 'lightgreen', 'salmon']):
        patch.set_facecolor(color)
    ax1.set_ylabel('Latency (μs)')
    ax1.set_title('Static Topology Performance')
    ax1.grid(True, alpha=0.3)

    # 2. OptiMoE Latency Evolution
    ax2 = fig.add_subplot(gs[0, 1:])
    iterations = [r['iteration'] for r in optimoe_results]
    latencies = [r['latency'] for r in optimoe_results]

    # Color by topology
    colors = {'fattree': 'blue', 'mesh': 'green', 'torus': 'red'}
    for i in range(len(iterations)-1):
        topo = optimoe_results[i]['topology']
        ax2.plot([iterations[i], iterations[i+1]],
                [latencies[i], latencies[i+1]],
                color=colors[topo], alpha=0.7, linewidth=2)

    # Mark reconfigurations
    reconfig_points = [i for i, r in enumerate(optimoe_results) if r['reconfigure']]
    if reconfig_points:
        reconfig_lats = [latencies[i] for i in reconfig_points]
        ax2.scatter([iterations[i] for i in reconfig_points],
                   reconfig_lats, color='black', s=50, zorder=5,
                   marker='v', label='Reconfiguration')

    ax2.set_xlabel('Iteration')
    ax2.set_ylabel('Latency (μs)')
    ax2.set_title('OptiMoE Dynamic Adaptation')
    ax2.legend()
    ax2.grid(True, alpha=0.3)

    # 3. Traffic Pattern Evolution
    ax3 = fig.add_subplot(gs[1, :])
    concentrations = [r['traffic_concentration'] for r in optimoe_results]
    localities = [r['traffic_locality'] for r in optimoe_results]
    variances = [r['traffic_variance'] for r in optimoe_results]

    ax3.plot(iterations, concentrations, 'b-', label='Concentration', alpha=0.7)
    ax3.plot(iterations, localities, 'g-', label='Locality', alpha=0.7)
    ax3.plot(iterations, variances, 'r-', label='Variance', alpha=0.7)

    # Shade regions by pattern
    pattern_changes = [0, 15, 30, 45, 60, 75, 90]
    pattern_names = ['Hotspot', 'Uniform', 'Regional', 'Skewed'] * 10
    for i in range(len(pattern_changes)-1):
        start = pattern_changes[i]
        end = pattern_changes[i+1] if i+1 < len(pattern_changes) else 100
        pattern = pattern_names[i % 4]
        color = ['pink', 'lightblue', 'lightgreen', 'lightyellow'][i % 4]
        ax3.axvspan(start, end, alpha=0.2, color=color)
        ax3.text((start+end)/2, 0.9, pattern, ha='center', fontsize=8)

    ax3.set_xlabel('Iteration')
    ax3.set_ylabel('Traffic Characteristics')
    ax3.set_title('Traffic Pattern Evolution')
    ax3.legend(loc='lower right')
    ax3.grid(True, alpha=0.3)

    # 4. Topology Usage Timeline
    ax4 = fig.add_subplot(gs[2, 0:2])

    # Create timeline visualization
    topology_timeline = []
    current_topo = optimoe_results[0]['topology']
    start_idx = 0

    for i, result in enumerate(optimoe_results):
        if result['topology'] != current_topo or i == len(optimoe_results)-1:
            topology_timeline.append({
                'topology': current_topo,
                'start': start_idx,
                'end': i
            })
            current_topo = result['topology']
            start_idx = i

    # Plot timeline bars
    y_pos = 0
    for segment in topology_timeline:
        width = segment['end'] - segment['start']
        color = colors[segment['topology']]
        ax4.barh(y_pos, width, left=segment['start'], color=color,
                alpha=0.7, edgecolor='black', linewidth=1)

    ax4.set_xlim(0, len(optimoe_results))
    ax4.set_ylim(-0.5, 0.5)
    ax4.set_xlabel('Iteration')
    ax4.set_yticks([])
    ax4.set_title('Topology Selection Timeline')

    # Add legend
    from matplotlib.patches import Patch
    legend_elements = [Patch(facecolor=colors[t], label=t) for t in colors.keys()]
    ax4.legend(handles=legend_elements, loc='upper right')

    # 5. Performance Metrics
    ax5 = fig.add_subplot(gs[2, 2])

    # Calculate metrics
    baseline_avg = np.mean([lat for results in baseline_data for lat in results])
    optimoe_avg = np.mean(latencies)
    improvement = (baseline_avg - optimoe_avg) / baseline_avg * 100

    reconfigs = sum(1 for r in optimoe_results if r['reconfigure'])
    reconfig_rate = reconfigs / len(optimoe_results) * 100

    # Topology usage
    topo_usage = defaultdict(int)
    for r in optimoe_results:
        topo_usage[r['topology']] += 1

    # Create metrics text
    metrics_text = f"""Performance Summary
    {'='*25}

    Baseline Avg: {baseline_avg:.1f} μs
    OptiMoE Avg:  {optimoe_avg:.1f} μs
    Improvement:  {improvement:+.1f}%

    Reconfigurations: {reconfigs}
    Switch Rate: {reconfig_rate:.1f}%

    Topology Usage:
    """

    for topo, count in topo_usage.items():
        pct = count / len(optimoe_results) * 100
        metrics_text += f"\n  {topo}: {pct:.1f}%"

    ax5.text(0.1, 0.9, metrics_text, transform=ax5.transAxes,
            fontsize=10, verticalalignment='top', fontfamily='monospace')
    ax5.axis('off')

    plt.suptitle('OptiMoE: Dynamic Topology-Aware Scheduling Results',
                fontsize=14, fontweight='bold')
    plt.tight_layout()
    return fig

# Generate visualization
fig = visualize_optimoe_results(baseline_results, optimoe_results)
plt.show()

In [ ]:
# Cell 6: Detailed Performance Analysis
"""Analyze switching decisions and their effectiveness"""

def analyze_switching_decisions(optimoe_results):
    """Analyze the effectiveness of topology switches"""

    switches = []
    for i, result in enumerate(optimoe_results):
        if result['reconfigure']:
            # Look at performance before and after switch
            before_window = 5
            after_window = 5

            before_start = max(0, i - before_window)
            after_end = min(len(optimoe_results), i + after_window)

            before_latencies = [optimoe_results[j]['latency']
                              for j in range(before_start, i)]
            after_latencies = [optimoe_results[j]['latency']
                             for j in range(i, after_end)]

            if before_latencies and after_latencies:
                before_avg = np.mean(before_latencies)
                after_avg = np.mean(after_latencies)
                improvement = (before_avg - after_avg) / before_avg * 100

                switches.append({
                    'iteration': i,
                    'from_topology': optimoe_results[i-1]['topology'] if i > 0 else 'unknown',
                    'to_topology': result['topology'],
                    'before_latency': before_avg,
                    'after_latency': after_avg,
                    'improvement': improvement,
                    'traffic_concentration': result['traffic_concentration'],
                    'traffic_locality': result['traffic_locality']
                })

    # Print analysis
    print("Topology Switch Analysis")
    print("="*60)

    for i, switch in enumerate(switches):
        print(f"\nSwitch {i+1} at iteration {switch['iteration']}:")
        print(f"  {switch['from_topology']} → {switch['to_topology']}")
        print(f"  Before: {switch['before_latency']:.1f} μs")
        print(f"  After:  {switch['after_latency']:.1f} μs")
        print(f"  Improvement: {switch['improvement']:+.1f}%")
        print(f"  Traffic: concentration={switch['traffic_concentration']:.2f}, "
              f"locality={switch['traffic_locality']:.2f}")

    # Overall statistics
    if switches:
        avg_improvement = np.mean([s['improvement'] for s in switches])
        successful_switches = sum(1 for s in switches if s['improvement'] > 0)

        print(f"\nOverall Switch Statistics:")
        print(f"  Total switches: {len(switches)}")
        print(f"  Successful switches: {successful_switches}/{len(switches)} "
              f"({successful_switches/len(switches)*100:.1f}%)")
        print(f"  Average improvement: {avg_improvement:+.1f}%")

    return switches

# Analyze switching decisions
switch_analysis = analyze_switching_decisions(optimoe_results)

In [ ]:
# Cell 7: Export Results for Paper
"""Export results in format suitable for paper"""

def export_paper_results(baseline_results, optimoe_results, switch_analysis):
    """Export results for paper"""

    # Calculate key metrics
    baseline_latencies = []
    for topo, results in baseline_results.items():
        baseline_latencies.extend([r['average_latency'] for r in results])

    optimoe_latencies = [r['latency'] for r in optimoe_results]

    baseline_avg = np.mean(baseline_latencies)
    optimoe_avg = np.mean(optimoe_latencies)
    improvement = (baseline_avg - optimoe_avg) / baseline_avg * 100

    # Create results dictionary
    paper_results = {
        'configuration': {
            'num_nodes': 64,
            'num_experts': 128,
            'num_iterations': len(optimoe_results)
        },
        'baseline': {
            'average_latency': baseline_avg,
            'std_latency': np.std(baseline_latencies),
            'min_latency': np.min(baseline_latencies),
            'max_latency': np.max(baseline_latencies)
        },
        'optimoe': {
            'average_latency': optimoe_avg,
            'std_latency': np.std(optimoe_latencies),
            'min_latency': np.min(optimoe_latencies),
            'max_latency': np.max(optimoe_latencies),
            'num_switches': len(switch_analysis),
            'switch_rate': len(switch_analysis) / len(optimoe_results) * 100
        },
        'improvement': {
            'percentage': improvement,
            'absolute_μs': baseline_avg - optimoe_avg
        },
        'topology_usage': {},
        'switch_effectiveness': {
            'average_improvement': np.mean([s['improvement'] for s in switch_analysis]) if switch_analysis else 0,
            'successful_rate': sum(1 for s in switch_analysis if s['improvement'] > 0) / len(switch_analysis) * 100 if switch_analysis else 0
        }
    }

    # Calculate topology usage
    for r in optimoe_results:
        topo = r['topology']
        if topo not in paper_results['topology_usage']:
            paper_results['topology_usage'][topo] = 0
        paper_results['topology_usage'][topo] += 1

    # Convert to percentages
    total = len(optimoe_results)
    for topo in paper_results['topology_usage']:
        count = paper_results['topology_usage'][topo]
        paper_results['topology_usage'][topo] = {
            'count': count,
            'percentage': count / total * 100
        }

    # Save to JSON
    with open('optimoe_paper_results.json', 'w') as f:
        json.dump(paper_results, f, indent=2)

    # Print LaTeX-ready table
    print("\nLaTeX Table for Paper:")
    print("="*60)
    print(r"\begin{table}[h]")
    print(r"\centering")
    print(r"\caption{OptiMoE Performance Results}")
    print(r"\begin{tabular}{lcc}")
    print(r"\toprule")
    print(r"Metric & Baseline & OptiMoE \\")
    print(r"\midrule")
    print(f"Average Latency (μs) & {baseline_avg:.1f} & {optimoe_avg:.1f} \\\\")
    print(f"Std. Deviation (μs) & {np.std(baseline_latencies):.1f} & {np.std(optimoe_latencies):.1f} \\\\")
    print(f"Min Latency (μs) & {np.min(baseline_latencies):.1f} & {np.min(optimoe_latencies):.1f} \\\\")
    print(f"Max Latency (μs) & {np.max(baseline_latencies):.1f} & {np.max(optimoe_latencies):.1f} \\\\")
    print(r"\midrule")
    print(f"Improvement & - & {improvement:+.1f}\\% \\\\")
    print(f"Reconfigurations & - & {len(switch_analysis)} \\\\")
    print(f"Switch Rate & - & {len(switch_analysis)/len(optimoe_results)*100:.1f}\\% \\\\")
    print(r"\bottomrule")
    print(r"\end{tabular}")
    print(r"\end{table}")

    return paper_results

# Export results
paper_results = export_paper_results(baseline_results, optimoe_results, switch_analysis)
print(f"\n✅ Results exported to 'optimoe_paper_results.json'")


In [ ]:
# Cell 8: Scalability Test
"""Test OptiMoE scalability with different cluster sizes"""

def test_scalability():
    """Test OptiMoE with different cluster sizes"""

    cluster_sizes = [16, 32, 64, 128]
    results = []

    print("Scalability Test")
    print("="*40)

    for size in cluster_sizes:
        print(f"\nTesting {size} nodes...")

        # Create simulation
        sim = OptiMoENetworkSimulation()
        sim.moe_config.num_nodes = size
        sim.moe_config.num_experts = size * 4

        # Run short experiments
        baseline = sim.run_baseline_comparison(num_iterations=5)
        optimoe = sim.run_optimoe_experiment(num_iterations=40)

        # Calculate metrics
        baseline_avg = np.mean([r['average_latency']
                               for results in baseline.values()
                               for r in results])
        optimoe_avg = np.mean([r['latency'] for r in optimoe])
        improvement = (baseline_avg - optimoe_avg) / baseline_avg * 100
        switches = sum(1 for r in optimoe if r['reconfigure'])

        results.append({
            'size': size,
            'baseline': baseline_avg,
            'optimoe': optimoe_avg,
            'improvement': improvement,
            'switches': switches
        })

        print(f"  Baseline: {baseline_avg:.1f} μs")
        print(f"  OptiMoE:  {optimoe_avg:.1f} μs")
        print(f"  Improvement: {improvement:+.1f}%")

    # Plot scaling results
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

    sizes = [r['size'] for r in results]
    improvements = [r['improvement'] for r in results]
    switches = [r['switches'] for r in results]

    ax1.plot(sizes, improvements, 'o-', linewidth=2, markersize=8)
    ax1.set_xlabel('Cluster Size (nodes)')
    ax1.set_ylabel('Performance Improvement (%)')
    ax1.set_title('OptiMoE Scalability')
    ax1.grid(True, alpha=0.3)
    ax1.set_xscale('log', base=2)

    ax2.plot(sizes, switches, 'o-', linewidth=2, markersize=8, color='orange')
    ax2.set_xlabel('Cluster Size (nodes)')
    ax2.set_ylabel('Number of Topology Switches')
    ax2.set_title('Reconfiguration Frequency')
    ax2.grid(True, alpha=0.3)
    ax2.set_xscale('log', base=2)

    plt.tight_layout()
    plt.show()

    return results

# Run scalability test (optional - takes time)
scalability_results = test_scalability()
print("Scalability test ready. Uncomment the line above to run.")